<a href="https://colab.research.google.com/github/nachomendoza97/Data-Industrial/blob/main/APP_OPT_Sheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
##Codigo de vinculacion de aplicaciones con plantillas de Google Sheets "Gestion de datos Industrial"
# --- 1. CONFIGURACIÓN Y AUTENTICACIÓN ---
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# --- 2. CONEXIÓN AL ARCHIVO (Usa tu ID de la imagen) ---
ID_HOJA = "1aPsyywIlGXf7R_GONxx8CCCcqvS4gEenTWHtylUVUA8"
wb = gc.open_by_key(ID_HOJA)
sheet_inp = wb.worksheet("INP_OPT-Scheduling")

def app_scheduling_opt():
    # A. Lectura de Matriz (Rango A1:K11)
    # Se eliminaron los espacios extra para evitar el IndentationError
    datos_matriz = sheet_inp.get('A1:K11')
    matriz_df = pd.DataFrame(datos_matriz[1:], columns=datos_matriz[0]).set_index('Lote')
    matriz_df.columns = matriz_df.columns.str.strip()
    matriz_df.index = matriz_df.index.str.strip()
    matriz_df = matriz_df.replace('N/A', 0).astype(float)

    # B. Lectura de Lotes (Rango A14:G25)
    datos_lotes = sheet_inp.get('A14:G25')
    df_lotes = pd.DataFrame(datos_lotes[1:], columns=datos_lotes[0])
    df_lotes.columns = df_lotes.columns.str.strip() # Previene KeyError

    # C. Conversión de Tipos
    df_lotes['Tiempo de Procesamiento (min)'] = pd.to_numeric(df_lotes['Tiempo de Procesamiento (min)'])
    df_lotes['# Tiempo de Set Up (min)'] = pd.to_numeric(df_lotes['# Tiempo de Set Up (min)'])

    # D. Lógica SPT + Short Tail
    df_ordenado = df_lotes.sort_values(
        by=['Tiempo de Procesamiento (min)', '# Tiempo de Set Up (min)'],
        ascending=[True, True]
    ).reset_index(drop=True)

    cronograma = []
    reloj = 0
    lote_prev = None

    for _, fila in df_ordenado.iterrows():
        lote_id = fila['Lote ID']
        co = 0 if lote_prev is None else matriz_df.loc[lote_prev, lote_id]

        inicio = reloj + co
        duracion = fila['Tiempo de Procesamiento (min)'] + fila['# Tiempo de Set Up (min)']
        fin = inicio + duracion

        cronograma.append({
            'Lote ID': lote_id,
            'ChangeOver_min': co,
            'Inicio_min': inicio,
            'Fin_min': fin
        })
        reloj = fin
        lote_prev = lote_id

    # E. Exportar a "Resultado_Gantt"
    df_res = pd.DataFrame(cronograma)
    res_sheet = wb.worksheet("Resultado_Gantt")
    res_sheet.clear()
    res_sheet.update([df_res.columns.values.tolist()] + df_res.values.tolist())
    print("✅ Programación de producción exportada exitosamente.")

# --- 3. EJECUCIÓN (Llamada final) ---
app_scheduling_opt()

SpreadsheetNotFound: <Response [404]>

In [9]:
datos_lotes = sheet_inp.get('A14:G25')
df_lotes_temp = pd.DataFrame(datos_lotes[1:], columns=datos_lotes[0])
print('Columnas encontradas en la hoja de cálculo:')
print(df_lotes_temp.columns.tolist())

Columnas encontradas en la hoja de cálculo:
['Lote ID', 'Producto/SKU', 'Tiempo de Procesamiento (min)', 'Tiempo de Set Up (min)', 'Rendimiento por Hora (Unidades)', 'Máquina/Línea']


In [3]:
import gspread
from google.colab import auth
from google.auth import default

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Reemplaza 'TU_ID_DE_HOJA_DE_CALCULO' con el ID real de tu hoja de cálculo
spreadsheet_id = 'TU_ID_DE_HOJA_DE_CALCULO'
wb = gc.open_by_id(spreadsheet_id)

# Continúa con el resto de tu código, por ejemplo:
sheet_inp = wb.worksheet("INP_OPT-Scheduling")
print("Conexión exitosa a la hoja de cálculo por ID.")

AttributeError: 'Client' object has no attribute 'open_by_id'

Recuerda que deberás reemplazar `TU_ID_DE_HOJA_DE_CALCULO` con el ID real de tu hoja de cálculo.